### Installing and importing libraries

In [1]:
!pip install hazm
!pip install pandas
!pip install numpy
!pip install nltk
!pip install tqdm
# !pip install transformers
!pip install fasttext

In [2]:
import pandas as pd
import numpy as np
from hazm import *
import tqdm
import json
import codecs
from nltk import FreqDist
import pickle
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# from transformers import BigBirdModel, AutoTokenizer
import fasttext

### Import data

In [4]:
news_dict = {}
with open('../data/news.json', "r", encoding="utf-8") as text_file:
        news_dict = json.loads(text_file.read())

df = pd.DataFrame.from_dict(news_dict, orient='index')
df

,title,subject,content
0,هاشمی: هماهنگ کردم آزمون خبرگان را از حسن روحا...,سیاسی,به گزارش همشهری آنلاین، روزنامه فرهیختگان در ش...
1,اصلاح قانون اسلحه از کجا کلید خورد؟ ا مامور ...,سیاسی,همشهری‌آنلاین- امیرمحمدحسینی:اصلاحیه قانون به ...
2,ایران در برابر قطعنامه‌های آژانس سکوت نمی‌کند ...,سیاسی,به گزارش همشهری‌آنلاین،فدا حسین مالکی در مورد ...
3,اینفوگرافیک | اصلاحات قانون به‌کارگیری اسلحه چ...,سیاسی,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ - Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ Ш...
4,پیام تبریک رئیسی به رییس‌جمهور فیلیپین,سیاسی,به گزارش همشهری‌آنلاین، آیت‌الله سید ابراهیم ر...
...,...,...,...
14871,عکس | ۲ ستاره تیم ملی در میان برترین گلزنان مق...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...
14872,عکس| ۲۷ تیم راه یافته به جام جهانی | ۵ تیم باق...,ورزش,به گزارش همشهری آنلاین با صعود پرتغال و لهستان...
14873,عکس | برد پرگل برزیل در شب خداحافظی کلمبیا و ش...,ورزش,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...
14874,خداحافظی غم انگیز کی روش با تیم ملی مصر | رنج ...,ورزش,به گزارش همشهری آنلاین تیم ملی مصر در مصاف با ...


### Pre processing

In [5]:
normalizer = Normalizer()

normalized_news_dict = {}
for idx,value in tqdm.tqdm(news_dict.items()):
    normalized_news_dict[idx] = {"title" : value['title'] , "content":normalizer.normalize(value['content'])}
  
news_df = pd.DataFrame.from_dict(normalized_news_dict, orient='index')

100%|███████████████████████████████████| 14876/14876 [00:12<00:00, 1144.81it/s]


In [6]:
new_stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('../custom_requirments/new_stopwords.txt','r','utf-8').readlines()]
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('../custom_requirments/stopwords.txt','r','utf-8').readlines()]

In [7]:
tokenized_words = [word_tokenize(_) for _ in news_df.content]

In [8]:
removed_tokenized_words = []
for per_content_words in tokenized_words:
  words = []
  for word in per_content_words:
    if word not in stopwords + new_stopwords:
      words.append(word)
  removed_tokenized_words.append(words)

news_df['word_tokenize'] = removed_tokenized_words
news_df

,title,content,word_tokenize
0,هاشمی: هماهنگ کردم آزمون خبرگان را از حسن روحا...,به گزارش همشهری آنلاین، روزنامه فرهیختگان در ش...,"[گزارش, همشهری, آنلاین, روزنامه, فرهیختگان, شم..."
1,اصلاح قانون اسلحه از کجا کلید خورد؟ ا مامور ...,همشهری‌آنلاین- امیرمحمدحسینی: اصلاحیه قانون به...,"[همشهری‌آنلاین-, امیرمحمدحسینی, اصلاحیه, قانون..."
2,ایران در برابر قطعنامه‌های آژانس سکوت نمی‌کند ...,به گزارش همشهری‌آنلاین، فدا حسین مالکی در مورد...,"[گزارش, همشهری‌آنلاین, فدا, حسین, مالکی, قطعنا..."
3,اینفوگرافیک | اصلاحات قانون به‌کارگیری اسلحه چ...,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ - Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ Ш...,"[ЩҮЩ…ШҙЩҮШұЫҢ, ШўЩҶЩ„Ш§ЫҢЩҶ, Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ,..."
4,پیام تبریک رئیسی به رییس‌جمهور فیلیپین,به گزارش همشهری‌آنلاین، آیت‌الله سید ابراهیم ر...,"[گزارش, همشهری‌آنلاین, آیت‌الله, سید, ابراهیم,..."
...,...,...,...
14871,عکس | ۲ ستاره تیم ملی در میان برترین گلزنان مق...,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"[گزارش, همشهری, آنلاین, مسابقات, مقدماتی, جام,..."
14872,عکس| ۲۷ تیم راه یافته به جام جهانی | ۵ تیم باق...,به گزارش همشهری آنلاین با صعود پرتغال و لهستان...,"[گزارش, همشهری, آنلاین, صعود, پرتغال, لهستان, ..."
14873,عکس | برد پرگل برزیل در شب خداحافظی کلمبیا و ش...,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"[گزارش, همشهری, آنلاین, مسابقات, مقدماتی, جام,..."
14874,خداحافظی غم انگیز کی روش با تیم ملی مصر | رنج ...,به گزارش همشهری آنلاین تیم ملی مصر در مصاف با ...,"[گزارش, همشهری, آنلاین, تیم, ملی, مصر, مصاف, س..."


In [9]:
def create_dataframe_least_freq(news_dict):
    news_tokens = {}
    for idx, value in tqdm.tqdm(news_dict.items()):
        news_tokens[idx] = FreqDist(value).most_common()[-10:]
    df = pd.DataFrame.from_dict(news_tokens, orient='index')
    return df.transpose()

def create_dataframe_most_freq(news_dict):
    news_tokens = {}
    for idx, value in tqdm.tqdm(news_dict.items()):
        news_tokens[idx] = FreqDist(value).most_common(10)
    df = pd.DataFrame.from_dict(news_tokens, orient='index')
    return df.transpose()

In [10]:
tokenized_news = {}
for idx,news in news_df.iterrows():
  tokenized_news[idx] = news.word_tokenize

least_freq_df = create_dataframe_least_freq(tokenized_news)
most_freq_df = create_dataframe_most_freq(tokenized_news)

100%|███████████████████████████████████| 14876/14876 [00:01<00:00, 7553.08it/s]


In [11]:
stemmer = Stemmer()
lemmatizer = Lemmatizer()

def get_lemma_set(tok, opt=1):
    if opt == 1:
        return stemmer.stem(tok)
    if opt == 2:
        return lemmatizer.lemmatize(tok)

In [12]:
opt = 2
lemmatize_stemming_words = []
for per_content_words in news_df.word_tokenize:
  words = []
  for word in per_content_words:
    words.append(get_lemma_set(word,opt))
  lemmatize_stemming_words.append(words)
news_df['word_lemmatize'] = lemmatize_stemming_words

In [13]:
words = []
persian_numbers = ['۱','۲','۳','۴','۵','۶','۷','۸','۹']
for row_words in news_df['word_lemmatize']:
    final_words = ','.join(row_words)
    final_words = final_words.replace('-','')
    for persian_num in persian_numbers:
        if persian_num in list(final_words):
            final_words.replace(persian_num, '')     
    words.append(final_words)
news_df['clean_keyword'] = words
news_df = news_df.drop(['word_tokenize','word_lemmatize'],axis=1)
news_df

,title,content,clean_keyword
0,هاشمی: هماهنگ کردم آزمون خبرگان را از حسن روحا...,به گزارش همشهری آنلاین، روزنامه فرهیختگان در ش...,"گزارش,همشهری,آنلاین,روزنامه,فرهیختگان,شماره,ام..."
1,اصلاح قانون اسلحه از کجا کلید خورد؟ ا مامور ...,همشهری‌آنلاین- امیرمحمدحسینی: اصلاحیه قانون به...,"همشهری‌آنلاین,امیرمحمدحسینی,اصلاحیه,قانون,کارگ..."
2,ایران در برابر قطعنامه‌های آژانس سکوت نمی‌کند ...,به گزارش همشهری‌آنلاین، فدا حسین مالکی در مورد...,"گزارش,همشهری‌آنلاین,فدا,حسین,مالکی,قطعنامه,ضد,..."
3,اینفوگرافیک | اصلاحات قانون به‌کارگیری اسلحه چ...,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ - Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ Ш...,"ЩҮЩ…ШҙЩҮШұЫҢ,ШўЩҶЩ„Ш§ЫҢЩҶ,Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ,ШӯШ..."
4,پیام تبریک رئیسی به رییس‌جمهور فیلیپین,به گزارش همشهری‌آنلاین، آیت‌الله سید ابراهیم ر...,"گزارش,همشهری‌آنلاین,آیت‌الله,سید,ابراهیم,رییس,..."
...,...,...,...
14871,عکس | ۲ ستاره تیم ملی در میان برترین گلزنان مق...,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"گزارش,همشهری,آنلاین,مسابقات,مقدمات,جام,جهانی,۲..."
14872,عکس| ۲۷ تیم راه یافته به جام جهانی | ۵ تیم باق...,به گزارش همشهری آنلاین با صعود پرتغال و لهستان...,"گزارش,همشهری,آنلاین,صعود,پرتغال,لهستان,۵,تیم,آ..."
14873,عکس | برد پرگل برزیل در شب خداحافظی کلمبیا و ش...,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"گزارش,همشهری,آنلاین,مسابقات,مقدمات,جام,جهانی,۲..."
14874,خداحافظی غم انگیز کی روش با تیم ملی مصر | رنج ...,به گزارش همشهری آنلاین تیم ملی مصر در مصاف با ...,"گزارش,همشهری,آنلاین,تیم,ملی,مصر,مصاف,سنگال,تسا..."


In [14]:
clean_keywords_non_least_frequent = []
for idx,per_final_news in news_df.iterrows():
  for jdx, per_freq in least_freq_df.iterrows():
    try:
      word = least_freq_df.iloc[jdx,int(idx)]
      removed = per_final_news.clean_keyword
      if word is not None:
        word = word[0]
        removed = removed.replace(word,'')
    except Exception as e:
      continue
  clean_keywords_non_least_frequent.append(removed)

clean_keywords_non_most_frequent = []
for idx,per_final_news in news_df.iterrows():
  for jdx, per_freq in most_freq_df.iterrows():
    try:
      word = most_freq_df.iloc[jdx,int(idx)]
      removed = per_final_news.clean_keyword
      if word is not None:
        word = word[0]
        removed = removed.replace(word,'')
    except Exception as e:
      continue
  clean_keywords_non_most_frequent.append(removed)

news_df['clean_keywords_non_least_freq'] = clean_keywords_non_least_frequent
news_df['clean_keywords_non_most_freq'] = clean_keywords_non_least_frequent
news_df

,title,content,clean_keyword,clean_keywords_non_least_freq,clean_keywords_non_most_freq
0,هاشمی: هماهنگ کردم آزمون خبرگان را از حسن روحا...,به گزارش همشهری آنلاین، روزنامه فرهیختگان در ش...,"گزارش,همشهری,آنلاین,روزنامه,فرهیختگان,شماره,ام...","گزارش,همشهری,آنلاین,روزنامه,فرهیختگان,شماره,ام...","گزارش,همشهری,آنلاین,روزنامه,فرهیختگان,شماره,ام..."
1,اصلاح قانون اسلحه از کجا کلید خورد؟ ا مامور ...,همشهری‌آنلاین- امیرمحمدحسینی: اصلاحیه قانون به...,"همشهری‌آنلاین,امیرمحمدحسینی,اصلاحیه,قانون,کارگ...","همشهری‌آنلاین,امیرمحمدحسینی,اصلاحیه,قانون,کارگ...","همشهری‌آنلاین,امیرمحمدحسینی,اصلاحیه,قانون,کارگ..."
2,ایران در برابر قطعنامه‌های آژانس سکوت نمی‌کند ...,به گزارش همشهری‌آنلاین، فدا حسین مالکی در مورد...,"گزارش,همشهری‌آنلاین,فدا,حسین,مالکی,قطعنامه,ضد,...","گزارش,همشهری‌آنلاین,فدا,حسین,مالکی,قطعنامه,ضد,...","گزارش,همشهری‌آنلاین,فدا,حسین,مالکی,قطعنامه,ضد,..."
3,اینفوگرافیک | اصلاحات قانون به‌کارگیری اسلحه چ...,ЩҮЩ…ШҙЩҮШұЫҢ ШўЩҶЩ„Ш§ЫҢЩҶ - Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ Ш...,"ЩҮЩ…ШҙЩҮШұЫҢ,ШўЩҶЩ„Ш§ЫҢЩҶ,Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ,ШӯШ...","ЩҮЩ…ШҙЩҮШұЫҢ,ШўЩҶЩ„Ш§ЫҢЩҶ,Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ,ШӯШ...","ЩҮЩ…ШҙЩҮШұЫҢ,ШўЩҶЩ„Ш§ЫҢЩҶ,Ш§Щ…ЫҢШұЩ…ШӯЩ…ШҜ,ШӯШ..."
4,پیام تبریک رئیسی به رییس‌جمهور فیلیپین,به گزارش همشهری‌آنلاین، آیت‌الله سید ابراهیم ر...,"گزارش,همشهری‌آنلاین,آیت‌الله,سید,ابراهیم,رییس,...","گزارش,همشهری‌آنلاین,آیت‌الله,سید,ابراهیم,رییس,...","گزارش,همشهری‌آنلاین,آیت‌الله,سید,ابراهیم,رییس,..."
...,...,...,...,...,...
14871,عکس | ۲ ستاره تیم ملی در میان برترین گلزنان مق...,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"گزارش,همشهری,آنلاین,مسابقات,مقدمات,جام,جهانی,۲...","گزارش,همشهری,آنلاین,مسابقات,مقدمات,جام,جهانی,۲...","گزارش,همشهری,آنلاین,مسابقات,مقدمات,جام,جهانی,۲..."
14872,عکس| ۲۷ تیم راه یافته به جام جهانی | ۵ تیم باق...,به گزارش همشهری آنلاین با صعود پرتغال و لهستان...,"گزارش,همشهری,آنلاین,صعود,پرتغال,لهستان,۵,تیم,آ...","گزارش,همشهری,آنلاین,صعود,پرتغال,لهستان,۵,تیم,آ...","گزارش,همشهری,آنلاین,صعود,پرتغال,لهستان,۵,تیم,آ..."
14873,عکس | برد پرگل برزیل در شب خداحافظی کلمبیا و ش...,به گزارش همشهری آنلاین مسابقات مقدماتی جام جها...,"گزارش,همشهری,آنلاین,مسابقات,مقدمات,جام,جهانی,۲...","گزارش,همشهری,آنلاین,مسابقات,مقدمات,جام,جهانی,۲...","گزارش,همشهری,آنلاین,مسابقات,مقدمات,جام,جهانی,۲..."
14874,خداحافظی غم انگیز کی روش با تیم ملی مصر | رنج ...,به گزارش همشهری آنلاین تیم ملی مصر در مصاف با ...,"گزارش,همشهری,آنلاین,تیم,ملی,مصر,مصاف,سنگال,تسا...","گزارش,همشهری,آنلاین,تیم,ملی,مصر,مصاف,سنگال,تسا...","گزارش,همشهری,آنلاین,تیم,ملی,مصر,مصاف,سنگال,تسا..."


### Inputs and constanst

In [15]:
DOC_RELATED_NUM = 10
query = 'استقلال پرسپولیس' 
query_words = query.split(" ")

### Query expansion

In [16]:
fasttext_model_loaded = fasttext.load_model('../models/QE_fasttext/fasttext.bin')

In [17]:
lines = ''
with open('../models/QE_fasttext/fasttext_data.txt') as f:
    lines = f.read()
words = lines.split(" ")

In [18]:
word2vec = {}
for word in words:
    word2vec[word] = fasttext_model_loaded[word]

In [19]:
import json
import numpy as np

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
        
with open('../models/QE_fasttext/all_words_vectors_emb_fasttext.json', 'w', encoding="utf-8") as f:
    json.dump(word2vec, f, cls=NumpyEncoder)

In [20]:
q_word2vec = {}
for q_word in query_words:
    q_word2vec[q_word] = fasttext_model_loaded[q_word]

In [21]:
with open('../models/QE_fasttext/all_words_vectors_emb_fasttext.json', 'r', encoding="utf-8") as f:
  word2vec_loaded = json.loads(f.read())

In [22]:
def expand_query(query_words, word2vec, cosine_threshold=0.9):
    most_similar_query = ''
    for q_word in query_words:
        max_cosine_similarity = 0
        most_similar_word = ''
        for k, v in word2vec.items():
            cosine_similarity = np.dot(np.array(v), np.array(q_word2vec[q_word])) / (np.linalg.norm(np.array(v)) * np.linalg.norm(np.array(q_word2vec[q_word])))
            k = k.replace("\u200e", "")
            k = k.replace("\xad","")
            k = k.replace("\n"," ")
            if cosine_similarity > max_cosine_similarity and k != q_word and cosine_similarity < cosine_threshold:
                max_cosine_similarity = cosine_similarity
                most_similar_word = k
        most_similar_query = most_similar_query + " " + most_similar_word
    return most_similar_query.strip()

In [23]:
def expand_query_rochio(q_avg, nr_avg, r_avg):
    temp_sum = np.sum([np.array(q_avg), r_avg],axis=0)
    new_q_avg = np.subtract(temp_sum, nr_avg)
    return new_q_avg

### Boolean retrieval

In [24]:
def create_boolean_retrival_matrix(df, query_words):
    boolean_retrival = [[0 for _ in range(len(query_words))] for __ in range(len(df))]
    for i in range(len(df)):
      keywords = df.iloc[i].clean_keyword
      for j,q_word in enumerate(query_words):
          if q_word in keywords.split(","):
              boolean_retrival[i][j] = 1
    boolean_df = pd.DataFrame(boolean_retrival, columns = query_words) 
    return boolean_df

def boolean_retirival_result(df):
    converted_df = boolean_df.all(axis='columns')
    converted_df = converted_df.to_frame('res')
    boolean_related_docs_index = converted_df.index[converted_df['res'] == True].tolist()
    boolean_related_titles = []
    for idx in boolean_related_docs_index:
      boolean_related_titles.append(news_df.iloc[idx,0])
    boolean_related_titles = list(dict.fromkeys(boolean_related_titles))
    return boolean_related_titles

def boolean_print_results(related, num=DOC_RELATED_NUM):
    for idx, title in enumerate(related[:num]):
          print(f"{idx+1}\t{title}")

def boolean_merge_results(qe_results, raw_results, num=DOC_RELATED_NUM):
    res = [*raw_results[:num], *qe_results[:num]]
    res = list(dict.fromkeys(res))
    for idx, title in enumerate(res):
          print(f"{idx+1}\t{title}")

In [25]:
boolean_df = create_boolean_retrival_matrix(news_df, query_words)
boolean_related_titles = boolean_retirival_result(boolean_df)
boolean_print_results(boolean_related_titles)

1	عکس | «رئیسی» پرسپولیسی‌ است یا استقلالی؟
2	تصمیم جدید برای بازپرداخت بدهی‌های سرخابی‌ها
3	نماد سرخابی‌ها چه زمانی بازگشایی می شود؟ | استقلال در ثبت افزایش سرمایه از پرسپولیس پیشی گرفت
4	سهامداران بخوانند | ۴ سیگنال جدید برای بورس
5	عرضه بلوک‌های مدیریتی سرخابی‌ها به کجا رسید؟
6	سهامداران بخوانند | ۴ سیگنال جدید برای بورسی‌ها
7	کاپیتان ۲۲ساله تیم ملی فوتبال در زمان شهادت لباس ورزشی برتن داشت | «مارادونا»ی ایران در روز تولدش، «شهادت» هدیه گرفت
8	حضور پیشکسوتان استقلال و پرسپولیس در مسابقات ورزشی شهرداری تهران
9	خدمات‌رسانی مترو به تماشاگران بازی استقلال و پرسپولیس | مترو در روز دربی تا ساعت 22  فعالیت می‌کند
10	فیلم و عکس | رقابت ستارگان هنر ایران برای آزادسازی زندانیان دیه | تلاش کامبیز دیرباز تا پویا امینی و علی لهراسبی برای کمک به حبسی‌های خاص


In [26]:
exapanded_query = expand_query(query_words, word2vec_loaded, 0.85)
qe_boolean_df = create_boolean_retrival_matrix(news_df, exapanded_query.split(" "))
qe_boolean_related_titles = boolean_retirival_result(qe_boolean_df)
boolean_print_results(qe_boolean_related_titles)

/var/folders/jl/n26nkxfn5fz5lng2zqgww5pr0000gn/T/ipykernel_2645/996594511.py:7: RuntimeWarning: invalid value encountered in double_scalars
  cosine_similarity = np.dot(np.array(v), np.array(q_word2vec[q_word])) / (np.linalg.norm(np.array(v)) * np.linalg.norm(np.array(q_word2vec[q_word])))


1	عکس | «رئیسی» پرسپولیسی‌ است یا استقلالی؟
2	تصمیم جدید برای بازپرداخت بدهی‌های سرخابی‌ها
3	نماد سرخابی‌ها چه زمانی بازگشایی می شود؟ | استقلال در ثبت افزایش سرمایه از پرسپولیس پیشی گرفت
4	سهامداران بخوانند | ۴ سیگنال جدید برای بورس
5	عرضه بلوک‌های مدیریتی سرخابی‌ها به کجا رسید؟
6	سهامداران بخوانند | ۴ سیگنال جدید برای بورسی‌ها
7	کاپیتان ۲۲ساله تیم ملی فوتبال در زمان شهادت لباس ورزشی برتن داشت | «مارادونا»ی ایران در روز تولدش، «شهادت» هدیه گرفت
8	حضور پیشکسوتان استقلال و پرسپولیس در مسابقات ورزشی شهرداری تهران
9	خدمات‌رسانی مترو به تماشاگران بازی استقلال و پرسپولیس | مترو در روز دربی تا ساعت 22  فعالیت می‌کند
10	فیلم و عکس | رقابت ستارگان هنر ایران برای آزادسازی زندانیان دیه | تلاش کامبیز دیرباز تا پویا امینی و علی لهراسبی برای کمک به حبسی‌های خاص


In [27]:
boolean_merge_results(qe_boolean_related_titles,boolean_related_titles)

1	عکس | «رئیسی» پرسپولیسی‌ است یا استقلالی؟
2	تصمیم جدید برای بازپرداخت بدهی‌های سرخابی‌ها
3	نماد سرخابی‌ها چه زمانی بازگشایی می شود؟ | استقلال در ثبت افزایش سرمایه از پرسپولیس پیشی گرفت
4	سهامداران بخوانند | ۴ سیگنال جدید برای بورس
5	عرضه بلوک‌های مدیریتی سرخابی‌ها به کجا رسید؟
6	سهامداران بخوانند | ۴ سیگنال جدید برای بورسی‌ها
7	کاپیتان ۲۲ساله تیم ملی فوتبال در زمان شهادت لباس ورزشی برتن داشت | «مارادونا»ی ایران در روز تولدش، «شهادت» هدیه گرفت
8	حضور پیشکسوتان استقلال و پرسپولیس در مسابقات ورزشی شهرداری تهران
9	خدمات‌رسانی مترو به تماشاگران بازی استقلال و پرسپولیس | مترو در روز دربی تا ساعت 22  فعالیت می‌کند
10	فیلم و عکس | رقابت ستارگان هنر ایران برای آزادسازی زندانیان دیه | تلاش کامبیز دیرباز تا پویا امینی و علی لهراسبی برای کمک به حبسی‌های خاص


### TF-IDF retrieval

In [28]:
def create_tf_idf_doc_term_matrix(df, tokenized_word):
    vocabulary = set()
    for doc in df.clean_keyword:
        vocabulary.update(doc.split(','))
    vocabulary = list(vocabulary)
    
    tfidf = TfidfVectorizer(vocabulary=vocabulary, use_idf = True, dtype=np.float32)
    tfidf_tran = tfidf.fit_transform([' '.join(doc) for doc in tokenized_word])
    return tfidf, tfidf_tran
    
def save_tf_idf(tfidf, tfidf_tran):
    with open('../models/tfidf_search/tfidf.pk', 'wb') as f:
        pickle.dump(tfidf, f)
    sparse.save_npz("../models/tfidf_search/tfidf_tran.npz", tfidf_tran)
    
def tf_idf_results(tf_idf, q_vec):
    return cosine_similarity(tf_idf, q_vec).reshape((-1, ))

def tf_idf_print_results(results):
    for idx,title in enumerate(results.argsort()[-10:][::-1]):
        print(f"{idx+1}\t{news_df.iloc[title,0]}")
        
def tf_idf_merge_results(qe_results, raw_results, num=DOC_RELATED_NUM):
    res = [*raw_results.argsort()[-10:][::-1], *qe_results.argsort()[-10:][::-1]]
    res = list(dict.fromkeys(res))
    for idx, title in enumerate(res):
          print(f"{idx+1}\t{news_df.iloc[title,0]}")

In [29]:
tfidf, tfidf_tran = create_tf_idf_doc_term_matrix(news_df, removed_tokenized_words)
save_tf_idf(tfidf, tfidf_tran)

/Users/nonaghazizadeh/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:1322: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [30]:
with open('../models/tfidf_search/tfidf.pk', 'rb') as f:
    tfidf_loaded = pickle.load(f)

tfidf_tran_loaded = sparse.load_npz("../models/tfidf_search/tfidf_tran.npz")

In [31]:
query_vec = tfidf_loaded.transform([query])
tfidf_results = tf_idf_results(tfidf_tran_loaded, query_vec)
tf_idf_print_results(tfidf_results)

1	حمله تند پیشکسوت پرسپولیس به گل‌محمدی | این اختلاف امتیاز واقعا شرم‌آور است | یحیی و دستیارانش همین حالا بروند
2	معاون جدید باشگاه پرسپولیس منصوب شد | گزینه ای که هواداران مخالف حضورش بودند
3	اولین خرید آبی‌ها در تابستان مشخص شد؟/ بازگشت مدافع سابق به استقلال!
4	جدایی دسته جمعی مربیان پرسپولیس | کوچ رفقای محبوب یحیی به تیم لیگ برتری
5	واکنش مدیرعامل باشگاه پیکان به اتهام تبانی | با واقعیت پرسپولیس باید کنار آمد
6	اولین اظهارات بازیکن جدید پرسپولیس | من را اکثر تیم های ایران می خواستند
7	مبلغ عجیب قرارداد رامین رضاییان با پرسپولیس | واکنش جالب مدافع محبوب به پیشنهاد استقلال و حضور در تیم ملی
8	سرگردانی استقلال با رفتن مجیدی | واکنش‌های منفی هواداران به گزینه ایرانی
9	فرهاد مجیدی رسماً از استقلال جدا شد | پیام خداحافظی سرمربی آبی‌ها؛ شک نکنید برمی‌گردم و تیم محبوبم را قهرمان آسیا می‌کنم
10	هافبک گلساز لیگ به پرسپولیس نزدیک شد | ستاره ملی‌پوش تیم فصل بعد خود را لو داد!


In [32]:
exapanded_query = expand_query(query_words, word2vec_loaded, 0.85)
qe_query_vec = tfidf_loaded.transform([exapanded_query])
qe_tfidf_results = tf_idf_results(tfidf_tran_loaded, qe_query_vec)
tf_idf_print_results(qe_tfidf_results)

/var/folders/jl/n26nkxfn5fz5lng2zqgww5pr0000gn/T/ipykernel_2645/996594511.py:7: RuntimeWarning: invalid value encountered in double_scalars
  cosine_similarity = np.dot(np.array(v), np.array(q_word2vec[q_word])) / (np.linalg.norm(np.array(v)) * np.linalg.norm(np.array(q_word2vec[q_word])))


1	خانه‌تکانی فرهاد شروع شد! | لیست احتمالی مجیدی برای خروج چند نفر از استقلال
2	توپ طلا؛ انگیزه جذاب هیولاها در پاریس | حمله به رکورد رونالدو در تکراری‌ترین فینال
3	سیدبندی ایران در جام جهانی مشخص شد | قرعه احتمالی برای تیم ملی؛ بازی قرن در انتظار شاگردان اسکوچیچ؟
4	پایان شوم تلاش پسر جوان برای ازدواج با دختر مورد علاقه‌اش
5	آخرین وضعیت نگهداری از توله یوز باقیمانده ایران | ایران نباید سزارین می‌شد
6	خودسوزی راننده شرکت دخانیات گلستان | علت حادثه در دست پیگیری است
7	«کوهستان» به نام پیر میدان‌های نبرد ثبت شد
8	سارق جوان موبایل راننده ال‌۹۰ را دزدید و کشته شد
9	ببینید | اقدام نجات‌بخش جوانان در بوستان‌ لاله
10	یک سوم ازدواج‌ها در شمال تهران در سال اول به طلاق می‌رسد | طلاق آسان در انتظار ازدواج‌های سخت! | ازدواج آسان موجب تقویت خانواده می‌شود


In [33]:
tf_idf_merge_results(qe_tfidf_results, tfidf_results)

1	حمله تند پیشکسوت پرسپولیس به گل‌محمدی | این اختلاف امتیاز واقعا شرم‌آور است | یحیی و دستیارانش همین حالا بروند
2	معاون جدید باشگاه پرسپولیس منصوب شد | گزینه ای که هواداران مخالف حضورش بودند
3	اولین خرید آبی‌ها در تابستان مشخص شد؟/ بازگشت مدافع سابق به استقلال!
4	جدایی دسته جمعی مربیان پرسپولیس | کوچ رفقای محبوب یحیی به تیم لیگ برتری
5	واکنش مدیرعامل باشگاه پیکان به اتهام تبانی | با واقعیت پرسپولیس باید کنار آمد
6	اولین اظهارات بازیکن جدید پرسپولیس | من را اکثر تیم های ایران می خواستند
7	مبلغ عجیب قرارداد رامین رضاییان با پرسپولیس | واکنش جالب مدافع محبوب به پیشنهاد استقلال و حضور در تیم ملی
8	سرگردانی استقلال با رفتن مجیدی | واکنش‌های منفی هواداران به گزینه ایرانی
9	فرهاد مجیدی رسماً از استقلال جدا شد | پیام خداحافظی سرمربی آبی‌ها؛ شک نکنید برمی‌گردم و تیم محبوبم را قهرمان آسیا می‌کنم
10	هافبک گلساز لیگ به پرسپولیس نزدیک شد | ستاره ملی‌پوش تیم فصل بعد خود را لو داد!
11	خانه‌تکانی فرهاد شروع شد! | لیست احتمالی مجیدی برای خروج چند نفر از استقلال
12	توپ طلا؛ انگیزه جذاب هیولاها در پاریس

### Transformers retrieval

In [63]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

def save_transformer_pretrained_model(path):
  MODEL_NAME = "SajjadAyoubi/distil-bigbird-fa-zwnj"
  model = BigBirdModel.from_pretrained(MODEL_NAME, block_size=32)
  tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
  model.save_pretrained(path + 'transformer_model.model')
  tokenizer.save_pretrained(path + 'transformer_tokenizer')

def load_transformer_pretrained_model(path):
  model = BigBirdModel.from_pretrained(path + 'transformer_model.model')
  tokenizer = AutoTokenizer.from_pretrained(path + 'transformer_tokenizer')
  return model, tokenizer

def create_doc_embedding_avg_transformer(tokenizer, model, df):
  docs_embedding_tran = {}
  for idx, doc in tqdm.tqdm(enumerate(df.clean_keyword)):
    text = ' '.join(doc.split(','))
    tokens = tokenizer(text, return_tensors='pt')
    try:
      output = model(**tokens)
      print(f"done{idx}")
    except Exception as e:
      text = ' '.join(doc.split(',')[:1000])
      tokens = tokenizer(text, return_tensors='pt')
      output = model(**tokens)
    docs_embedding_tran[idx] = output[0][0].detach().numpy()
    
  docs_embedding_tran_avg = {}
  for k,v in docs_embedding_tran.items():
    sum = np.zeros(768)
    for idx, word_embedding in enumerate(v):
      sum = np.sum([sum, docs_embedding_tran[k][idx]], axis=0)
    docs_embedding_tran_avg[k] = sum / len(docs_embedding_tran[k])
  return docs_embedding_tran_avg

def save_transformer_doc_embedding_avg(doc_embedding, path):
  with open(path, 'w', encoding="utf-8") as f:
    json.dump(doc_embedding, f, cls=NumpyEncoder)

def transformer_query_embedding(tokenizer, model, query):
  qtokens_tran = tokenizer(query, return_tensors='pt')
  qoutput_tran = model(**qtokens_tran)
  query_embedding_tran = qoutput_tran[0][0].detach().numpy()
  qsum_tran = np.zeros(768)
  for qidx,qword in enumerate(query_embedding_tran):
    qsum_tran = np.sum([qsum_tran, np.array(query_embedding_tran[qidx])], axis=0)
  q_avg_tran = qsum_tran / len(query_embedding_tran)
  return q_avg_tran

def transformers_results(doc_embeddings, query_embedding):
  docs_cosine_similarity_tran = {}
  for idx,doc_embedding in doc_embeddings.items():
    docs_cosine_similarity_tran[idx] = np.dot(doc_embedding, query_embedding) / (np.linalg.norm(doc_embedding)*np.linalg.norm(query_embedding))

  docs_cosine_similarity_tran = reversed(sorted(docs_cosine_similarity_tran.items(), key=lambda x:x[1]))
  sort_docs_cosine_similarity_tran = dict(docs_cosine_similarity_tran)
  return sort_docs_cosine_similarity_tran

def transformer_find_non_related_docs_avg(result, doc_embeddings, num=DOC_RELATED_NUM):
    sum = np.zeros(768)
    for idx, doc_id in enumerate((list(result.items()))[-10:]):
        sum = np.sum([sum, doc_embeddings[doc_id[0]]],axis=0)
    avg_non_related = sum / DOC_RELATED_NUM
    return avg_non_related

def transformer_find_related_docs_avg(result, doc_embeddings, num=DOC_RELATED_NUM):
    sum = np.zeros(768)
    for idx, doc_id in enumerate((list(result.items()))[:10]):
        sum = np.sum([sum, doc_embeddings[doc_id[0]]],axis=0)
    avg_related = sum / DOC_RELATED_NUM
    return avg_related

def transformer_print_results(result):
  for idx, doc_id in enumerate((list(result.items()))[:10]):
    print(f"{idx+1}   {news_df.iloc[int(doc_id[0])].title}")

def transformer_merge_results(qe_results, raw_results, num=DOC_RELATED_NUM):
    res = [*(list(qe_results.items()))[:num], *(list(raw_results.items()))[:num]]
    res = list(dict.fromkeys(res))
    for idx, doc_id in enumerate(res):
        print(f"{idx+1}   {news_df.iloc[int(doc_id[0])].title}")  

In [ ]:
save_transformer_pretrained_model('/content/models/')
model_loaded, tokenizer_loaded = load_transformer_pretrained_model('/content/models/') 
docs_embedding_tran_avg = create_doc_embedding_avg_transformer(tokenizer_loaded, model_loaded, news_df)
save_transformer_doc_embedding_avg(docs_embedding_tran_avg, '/content/models/transformer_vectors_emb.json')

In [48]:
with open('/content/models/transformer_vectors_emb.json', 'r', encoding="utf-8") as f:
  loaded_docs_embs_transformers = json.loads(f.read())

In [55]:
transformer_query_avg_tran = transformer_query_embedding(tokenizer_loaded, model_loaded, query)
transformer_results_cosine = transformers_results(loaded_docs_embs_transformers, transformer_query_avg_tran)
transformer_print_results(transformer_results_cosine)

1   رسمی| خرید جدید آبی ها رونمایی شد | مدافع لیگ برتری به استقلال پیوست
2   تصاویر لحظه بازداشت معاون سفیر انگلیس با چشمان بسته توسط اطلاعات سپاه
3   تعویق تمرینات پرسپولیس به زمان نامشخص
4   حضور آقای گل لیگ در پرسپولیس منتفی شد | منشا قراردادش را تمدید کرد
5   عکس| خرید خارجی پرسپولیس وارد تهران شد
6   رسمی | قرارداد بازیکن خارجی با پرسپولیس امضا شد
7   یک پرسپولیسی دیگر  رسما جدا شد | بازیکن سرخ‌پوش در تیم لیگ برتری
8   عکس | واکنش ستاره پرسپولیسی به ماجرای قراردادهای نجومی
9   زمان حضور خارجی‌های جدید پرسپولیس در تهران مشخص شد
10   اضافه شدن ازبک‌ها به رقابت سرخابی ها | پیشنهاد خارجی برای مدافع ملی‌پوش


In [64]:
nr_doc_avg_transformers = transformer_find_non_related_docs_avg(transformer_results_cosine, loaded_docs_embs_transformers)
r_doc_avg_transformers = transformer_find_related_docs_avg(transformer_results_cosine, loaded_docs_embs_transformers)
new_query_embedding_avg_transformers = expand_query_rochio(transformer_query_avg_tran, nr_doc_avg_transformers, r_doc_avg_transformers)
qe_transformers_results_cosines = transformers_results(loaded_docs_embs_transformers, new_query_embedding_avg_transformers)
transformer_print_results(qe_transformers_results_cosines)

1   حضور آقای گل لیگ در پرسپولیس منتفی شد | منشا قراردادش را تمدید کرد
2   رسمی| خرید جدید آبی ها رونمایی شد | مدافع لیگ برتری به استقلال پیوست
3   مهاجم خارجی پرسپولیس به تهران رسید
4   ورود مربی خارجی پرسپولیس به ایران
5   عکس | پست برانکو برای خداحافظی کاپیتان پرافتخار پرسپولیس
6   یک پرسپولیسی دیگر  رسما جدا شد | بازیکن سرخ‌پوش در تیم لیگ برتری
7   عکس| خرید خارجی پرسپولیس وارد تهران شد
8   زمان حضور خارجی‌های جدید پرسپولیس در تهران مشخص شد
9   رسمی | قرارداد بازیکن خارجی با پرسپولیس امضا شد
10   تعویق تمرینات پرسپولیس به زمان نامشخص


In [65]:
transformer_merge_results(qe_transformers_results_cosines, transformer_results_cosine)

1   حضور آقای گل لیگ در پرسپولیس منتفی شد | منشا قراردادش را تمدید کرد
2   رسمی| خرید جدید آبی ها رونمایی شد | مدافع لیگ برتری به استقلال پیوست
3   مهاجم خارجی پرسپولیس به تهران رسید
4   ورود مربی خارجی پرسپولیس به ایران
5   عکس | پست برانکو برای خداحافظی کاپیتان پرافتخار پرسپولیس
6   یک پرسپولیسی دیگر  رسما جدا شد | بازیکن سرخ‌پوش در تیم لیگ برتری
7   عکس| خرید خارجی پرسپولیس وارد تهران شد
8   زمان حضور خارجی‌های جدید پرسپولیس در تهران مشخص شد
9   رسمی | قرارداد بازیکن خارجی با پرسپولیس امضا شد
10   تعویق تمرینات پرسپولیس به زمان نامشخص
11   رسمی| خرید جدید آبی ها رونمایی شد | مدافع لیگ برتری به استقلال پیوست
12   تصاویر لحظه بازداشت معاون سفیر انگلیس با چشمان بسته توسط اطلاعات سپاه
13   تعویق تمرینات پرسپولیس به زمان نامشخص
14   حضور آقای گل لیگ در پرسپولیس منتفی شد | منشا قراردادش را تمدید کرد
15   عکس| خرید خارجی پرسپولیس وارد تهران شد
16   رسمی | قرارداد بازیکن خارجی با پرسپولیس امضا شد
17   یک پرسپولیسی دیگر  رسما جدا شد | بازیکن سرخ‌پوش در تیم لیگ برتری
18   عکس | واکنش ستاره پر

### Fasttext retrieval

In [18]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)
    
def create_fasttext_data(df):
    tokens_list = df.clean_keyword.tolist()
    all_news_content = ''
    for doc in tokens_list:
        all_news_content += ' '.join(doc.split(",")) + '\n'
    return all_news_content

def save_fasttext_data(data, path):
    with open(path, 'w') as text_file:
        text_file.write(data)

def train_fasttext(path):
    return fasttext.train_unsupervised(path, model='skipgram', minCount=4)

def save_fasttext(model, path):
    model.save_model(path)
    
def create_doc_embedding(df, model):
    docs_embedding_avg = {}
    for idx, doc in enumerate(df.clean_keyword):
        sum = np.zeros(100)
        for word in doc.split(','):
            sum = np.sum([sum, model[word]],axis=0)
        docs_embedding_avg[idx] = sum / len(doc.split(','))
    return docs_embedding_avg

def save_doc_embedding(doc_embedding, path):        
    with open(path, 'w', encoding="utf-8") as f:
        json.dump(doc_embedding, f, cls=NumpyEncoder)

def query_embedding(q_words, model):
    query_sum = np.zeros(100)
    for qword in q_words:
        query_sum = np.sum([query_sum, model[qword]], axis = 0)
    query_embedding_avg = query_sum / len(q_words)
    return query_embedding_avg

def fasttext_results(doc_embeddings, query_embedding):
    docs_cosine_similarity = {}
    for idx,doc_embedding in doc_embeddings.items():
        docs_cosine_similarity[idx] = np.dot(np.array(doc_embedding), query_embedding) / (np.linalg.norm(np.array(doc_embedding))*np.linalg.norm(query_embedding))

    docs_cosine_similarity = reversed(sorted(docs_cosine_similarity.items(), key=lambda x:x[1]))
    sort_docs_cosine_similarity = dict(docs_cosine_similarity)
    return sort_docs_cosine_similarity

def find_non_related_docs_avg(result, doc_embeddings, num=DOC_RELATED_NUM):
    sum = np.zeros(100)
    for idx, doc_id in enumerate((list(result.items()))[-10:]):
        sum = np.sum([sum, doc_embeddings[doc_id[0]]],axis=0)
    avg_non_related = sum / DOC_RELATED_NUM
    return avg_non_related

def find_related_docs_avg(result, doc_embeddings, num=DOC_RELATED_NUM):
    sum = np.zeros(100)
    for idx, doc_id in enumerate((list(result.items()))[:10]):
        sum = np.sum([sum, doc_embeddings[doc_id[0]]],axis=0)
    avg_related = sum / DOC_RELATED_NUM
    return avg_related

def fasttext_print_results(result):
    for idx, doc_id in enumerate((list(result.items()))[:10]):
        print(f"{idx+1}   {news_df.iloc[int(doc_id[0])].title}")
        
def fasttext_merge_results(qe_results, raw_results, num=DOC_RELATED_NUM):
    res = [*(list(qe_results.items()))[:num], *(list(raw_results.items()))[:num]]
    res = list(dict.fromkeys(res))
    for idx, doc_id in enumerate(res):
        print(f"{idx+1}   {news_df.iloc[int(doc_id[0])].title}")    

In [20]:
all_news_content = create_fasttext_data(news_df)
save_fasttext_data(all_news_content, '/content/models/fasttext_data.txt')
fasttext_model = train_fasttext('/content/models/fasttext_data.txt')
save_fasttext(fasttext_model, '/content/models/fasttext.bin')

In [21]:
fasttext_model_loaded = fasttext.load_model('/content/models/fasttext.bin')
docs_embedding_avg = create_doc_embedding(news_df, fasttext_model_loaded)
save_doc_embedding(docs_embedding_avg, '/content/models/fasttext_vectors_emb.json')
query_embedding_avg = query_embedding(query_words, fasttext_model_loaded)

In [22]:
with open('/content/models/fasttext_vectors_emb.json', 'r', encoding="utf-8") as f:
  loaded_docs_embs_ft = json.loads(f.read())

In [23]:
fasttext_results_cosines = fasttext_results(loaded_docs_embs_ft, query_embedding_avg)
fasttext_print_results(fasttext_results_cosines)

1   یک استقلالی دیگر هم تمدید کرد | هافبک آبی ها از لیست خروج بیرون آمد
2   رسمی| خیال آبی ها راحت شد | تمدید بلندمدت قرارداد ستاره خارجی استقلال
3   عکس | هافبک میانی استقلال هم تمدید کرد
4   حمله سپاهان به قهرمان لیگ | استقلال در آستانه حراج!
5   ریسک بازیکن استقلال با سیستم ساپینتو | تلخ و شیرین جشن آبی‌ها بسته به تصمیم یک ملی‌پوش!
6   مدافع آبی‌ها از استقلال مهلت خواست | قراردادی با میرزایی امضا نشده است
7   جدایی بازیکن پرسپولیس قطعی شد | برگه فسخ قرارداد آماده امضا
8   تغییر چهره استقلال با ساپینتو | قهرمان جوان‌تر می‌شود
9   یک بازیکن دیگر هم استقلال را ترک می‌کند | هافبک آبی‌ها رضایت‌نامه می‌خواهد
10   سپاهان آماده رونمایی از یک خرید بزرگ | ستاره محبوب استقلالی ها در آستانه زردپوش شدن


In [34]:
nr_doc_avg = find_non_related_docs_avg(fasttext_results_cosines, loaded_docs_embs_ft)
r_doc_avg = find_related_docs_avg(fasttext_results_cosines, loaded_docs_embs_ft)
new_query_embedding_avg = expand_query_rochio(query_embedding_avg, nr_doc_avg, r_doc_avg)
qe_fasttext_results_cosines = fasttext_results(loaded_docs_embs_ft, new_query_embedding_avg)
fasttext_print_results(qe_fasttext_results_cosines)

1   تعویق تمرینات پرسپولیس به زمان نامشخص
2   یک استقلالی دیگر هم تمدید کرد | هافبک آبی ها از لیست خروج بیرون آمد
3   کریم باقری به آمریکا رفت | استارت پرسپولیس برای فصل بعد بدون مربی محبوب
4   یک بازیکن دیگر هم استقلال را ترک می‌کند | هافبک آبی‌ها رضایت‌نامه می‌خواهد
5   ادامه طلسم تمرینات پیش فصل استقلال | تاخیر دوباره در استارت ساپینتو؟
6   اولین حریف پرسپولیس مشخص شد
7   بازگشت بزرگ یک ستاره به سمت استقلال | قرارداد گزینه محبوب هواداران به زودی امضا می شود
8   شروع متفاوت یحیی در پرسپولیس | از تیم من دور شوید
9   عکس | پایان دوران وریا غفوری در استقلال | کاپیتان محبوب آبی‌ها رسما خداحافظی کرد!
10   اقدام آبی‌ها برای حفظ کاپیتان استقلال | تصمیم احتمالی باشگاه مشخص شد


In [35]:
fasttext_merge_results(qe_fasttext_results_cosines, fasttext_results_cosines)

1   تعویق تمرینات پرسپولیس به زمان نامشخص
2   یک استقلالی دیگر هم تمدید کرد | هافبک آبی ها از لیست خروج بیرون آمد
3   کریم باقری به آمریکا رفت | استارت پرسپولیس برای فصل بعد بدون مربی محبوب
4   یک بازیکن دیگر هم استقلال را ترک می‌کند | هافبک آبی‌ها رضایت‌نامه می‌خواهد
5   ادامه طلسم تمرینات پیش فصل استقلال | تاخیر دوباره در استارت ساپینتو؟
6   اولین حریف پرسپولیس مشخص شد
7   بازگشت بزرگ یک ستاره به سمت استقلال | قرارداد گزینه محبوب هواداران به زودی امضا می شود
8   شروع متفاوت یحیی در پرسپولیس | از تیم من دور شوید
9   عکس | پایان دوران وریا غفوری در استقلال | کاپیتان محبوب آبی‌ها رسما خداحافظی کرد!
10   اقدام آبی‌ها برای حفظ کاپیتان استقلال | تصمیم احتمالی باشگاه مشخص شد
11   یک استقلالی دیگر هم تمدید کرد | هافبک آبی ها از لیست خروج بیرون آمد
12   رسمی| خیال آبی ها راحت شد | تمدید بلندمدت قرارداد ستاره خارجی استقلال
13   عکس | هافبک میانی استقلال هم تمدید کرد
14   حمله سپاهان به قهرمان لیگ | استقلال در آستانه حراج!
15   ریسک بازیکن استقلال با سیستم ساپینتو | تلخ و شیرین جشن آبی‌ها بسته 